In [4]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [6]:
from OAH_refocus import *
import matplotlib.pyplot as plt 
from OAHDEV_functions import *

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from scipy.signal import savgol_filter

import os
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
from IPython.display import display, HTML

import tkinter as tk
from tkinter import filedialog
from PIL import Image as PILImage

display(HTML("<style>.container { width:80% !important; }</style>"))
%matplotlib notebook

# To hide all the code
from IPython.display import HTML

def bin_image(arr, factor=4):
    h, w = arr.shape
    h_trim = (h // factor) * factor
    w_trim = (w // factor) * factor
    trimmed_arr = arr[:h_trim, :w_trim] 
    # Reshape and bin by averaging blocks
    binned_arr = trimmed_arr.reshape(h_trim // factor, factor, w_trim // factor, factor).mean(axis=(1, 3))
    return binned_arr

# Function to select multiple PNG files
def select_files(initial_dir="/home/bec_lab/Desktop/imgs/"):
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Ensure it stays on top
    file_selected = filedialog.askopenfilenames(
        title='Select PNG Files',
        initialdir=initial_dir,
        filetypes=[("PNG Files", "*.png")])
    root.destroy()  # Close the Tkinter window
    return file_selected if file_selected else None

# Function to create a GIF from selected images
def create_gif(image_files, output_gif="output.gif", frame_duration=100):
    if not image_files:
        print("No files selected.")
        return None

    images = [PILImage.open(file) for file in sorted(image_files)]

    output_dir = os.path.dirname(image_files[0])  # Save GIF in the same directory as the images
    output_path = os.path.join(output_dir, output_gif)

    images[0].save(
        output_path,
        save_all=True,
        append_images=images[1:],
        duration=frame_duration,
        loop=0
    )
    print(f"GIF saved as {output_path}")
    
    return output_path  # Return the path of the created GIF

def on_create_clicked(b):
    with output:
        output.clear_output()
        files = select_files()
        if files:
#             print(f"Selected Files: {files}")
            gif_path = create_gif(files)
            if gif_path:
                display(Image(filename=gif_path))  # Display the GIF
                button_close.disabled = False  # Enable close button
        else:
            print("No files selected.")

def on_close_clicked(b):
    with output:
        output.clear_output()
        print("GIF closed.")
        button_close.disabled = True  # Disable close button again 

In [7]:
# UI Elements
button_create = widgets.Button(description="Select Files and Create GIF")
button_close = widgets.Button(description="Close GIF", disabled=True)  # Disabled initially
output = widgets.Output()

button_create.on_click(on_create_clicked)
button_close.on_click(on_close_clicked)

# Display the buttons and output
display(button_create, button_close, output)


Button(description='Select Files and Create GIF', style=ButtonStyle())

Button(description='Close GIF', disabled=True, style=ButtonStyle())

Output()

In [ ]:
import os
import base64
import ipywidgets as widgets
from IPython.display import display, Image, clear_output, HTML
import tkinter as tk
from tkinter import filedialog
from PIL import Image as PILImage
import cv2

# Function to select multiple PNG files
def select_files(initial_dir="/home/bec_lab/Desktop/imgs/"):
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Ensure it stays on top
    file_selected = filedialog.askopenfilenames(
        title='Select PNG Files',
        initialdir=initial_dir,
        filetypes=[("PNG Files", "*.png")])
    root.destroy()  # Close the Tkinter window
    return file_selected if file_selected else None

# Function to create a GIF
def create_gif(image_files, output_name="output.gif", frame_duration=500):
    if not image_files:
        print("No files selected.")
        return None

    images = [PILImage.open(file) for file in sorted(image_files)]
    output_dir = os.path.dirname(image_files[0])
    output_path = os.path.join(output_dir, output_name)

    images[0].save(
        output_path,
        save_all=True,
        append_images=images[1:],
        duration=frame_duration,
        loop=0
    )
    print(f"GIF saved as {output_path}")
    
    return output_path

# Function to create an MP4 video
def create_mp4(image_files, output_name="output.mp4", frame_rate=10):
    if not image_files:
        print("No files selected.")
        return None

    output_dir = os.path.dirname(image_files[0])
    output_path = os.path.join(output_dir, output_name)

    images = sorted(image_files)
    first_image = cv2.imread(images[0])
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

    for img_file in images:
        img = cv2.imread(img_file)
        video.write(img)

    video.release()
    print(f"MP4 video saved as {output_path}")

    return output_path

# Function to convert MP4 to base64 (for Jupyter Notebook playback)
def encode_mp4_to_base64(video_path):
    with open(video_path, "rb") as video_file:
        video_base64 = base64.b64encode(video_file.read()).decode("utf-8")
    return f"data:video/mp4;base64,{video_base64}"

# UI Elements
button_create = widgets.Button(description="Select Files and Create")
button_close = widgets.Button(description="Close Output", disabled=True)
output = widgets.Output()
format_selector = widgets.RadioButtons(
    options=['GIF', 'MP4'],
    description='Format:',
    disabled=False
)

def on_create_clicked(b):
    with output:
        output.clear_output()
        files = select_files()
        if files:
            print(f"Selected Files: {files}")
            if format_selector.value == 'GIF':
                gif_path = create_gif(files)
                if gif_path:
                    display(Image(filename=gif_path))  # Display GIF
                    button_close.disabled = False
            else:
                mp4_path = create_mp4(files)
                if mp4_path:
                    video_data_url = encode_mp4_to_base64(mp4_path)
                    video_html = f"""
                    <video width="500" controls>
                        <source src="{video_data_url}" type="video/mp4">
                        Your browser does not support the video tag.
                    </video>
                    """
                    display(HTML(video_html))  # Display MP4 video
                    button_close.disabled = False
        else:
            print("No files selected.")

def on_close_clicked(b):
    with output:
        output.clear_output()
        print("Output closed.")
        button_close.disabled = True

button_create.on_click(on_create_clicked)
button_close.on_click(on_close_clicked)

# Display UI elements
display(format_selector, button_create, button_close, output)


In [ ]:
import os
import base64
import ipywidgets as widgets
from IPython.display import display, Image, clear_output, HTML
from PIL import Image as PILImage
import cv2

# File upload widget
file_upload = widgets.FileUpload(accept='.png', multiple=True)

# Function to get uploaded file paths
def get_uploaded_files(file_upload_widget):
    file_paths = []
    for filename, file_info in file_upload_widget.value.items():
        temp_path = f"/tmp/{filename}"
        with open(temp_path, "wb") as f:
            f.write(file_info['content'])
        file_paths.append(temp_path)
    return file_paths

# Function to create a GIF
def create_gif(image_files, output_name="output.gif", frame_duration=200):
    if not image_files:
        print("No files selected.")
        return None

    images = [PILImage.open(file) for file in sorted(image_files, reverse=False)]
    output_path = f"/tmp/{output_name}"

    images[0].save(
        output_path,
        save_all=True,
        append_images=images[1:],
        duration=frame_duration,
        loop=0
    )
    print(f"GIF saved as {output_path}")
    
    return output_path

# Function to create an MP4 video
def create_mp4(image_files, output_name="output.mp4", frame_rate=10):
    if not image_files:
        print("No files selected.")
        return None

    output_path = f"/tmp/{output_name}"

    images = sorted(image_files, reverse=False)
    first_image = cv2.imread(images[0])
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

    for img_file in images:
        img = cv2.imread(img_file)
        video.write(img)

    video.release()
    print(f"MP4 video saved as {output_path}")

    return output_path

# Function to convert MP4 to base64 (for Jupyter Notebook playback)
def encode_mp4_to_base64(video_path):
    with open(video_path, "rb") as video_file:
        video_base64 = base64.b64encode(video_file.read()).decode("utf-8")
    return f"data:video/mp4;base64,{video_base64}"

# UI Elements
button_create = widgets.Button(description="Create Animation")
button_close = widgets.Button(description="Close Output", disabled=True)
output = widgets.Output()
format_selector = widgets.RadioButtons(
    options=['GIF', 'MP4'],
    description='Format:',
    disabled=False
)

def on_create_clicked(b):
    with output:
        output.clear_output()
        files = get_uploaded_files(file_upload)
        if files:
            print(f"Selected Files: {files}")
            if format_selector.value == 'GIF':
                gif_path = create_gif(files)
                if gif_path:
                    display(Image(filename=gif_path))  # Display GIF
                    button_close.disabled = False
            else:
                mp4_path = create_mp4(files)
                if mp4_path:
                    video_data_url = encode_mp4_to_base64(mp4_path)
                    video_html = f"""
                    <video width="500" controls>
                        <source src="{video_data_url}" type="video/mp4">
                        Your browser does not support the video tag.
                    </video>
                    """
                    display(HTML(video_html))  # Display MP4 video
                    button_close.disabled = False
        else:
            print("No files uploaded.")

def on_close_clicked(b):
    with output:
        output.clear_output()
        print("Output closed.")
        button_close.disabled = True

button_create.on_click(on_create_clicked)
button_close.on_click(on_close_clicked)

# Display UI elements
display(file_upload, format_selector, button_create, button_close, output)
